In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime

Mounted at /content/drive/


#PE file

In [2]:
df1 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/DRF-80 - Brandon Myers 2024-25 PublicEnrollment 01292025.xlsx")

In [3]:
df1.head()

,SCHOOL_YEAR_ID,STN,IDOE_CORPORATION_ID,CORPORATION_NAME,IDOE_SCHOOL_ID,SCHOOL_NAME,BIRTH_DATE,PRIMARY_EXCEPTIONALITY_CODE,PRIMARY_EXCEPTIONALITY,SOCIO_ECON_STATUS,ETHNICITY
0,2025,002016067,15,Adams Central Community Schools,0013,Adams Central Middle School,01/06/2011,-9,NaN,Paid meals,White
1,2025,002016082,15,Adams Central Community Schools,0013,Adams Central Middle School,06/24/2011,-9,NaN,Paid meals,White
2,2025,002017020,15,Adams Central Community Schools,0013,Adams Central Middle School,10/29/2010,-9,NaN,Paid meals,White
3,2025,002017052,15,Adams Central Community Schools,0013,Adams Central Middle School,07/10/2010,-9,NaN,Paid meals,White
4,2025,002017056,15,Adams Central Community Schools,0013,Adams Central Middle School,08/02/2010,-9,NaN,Paid meals,Black


In [4]:
master_corp = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/corp_no_from_sig_dispro_calc.xlsx")

#### LEA from Sig dispor calculator





In [5]:
all_cops = master_corp['Reporting Corp'].unique()

In [6]:
#df1.PRIMARY_EXCEPTIONALITY.value_counts()

In [7]:
pd.set_option('display.max_columns', None)

In [8]:
#df1.head()

In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1040190 entries, 0 to 1040189
Data columns (total 11 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   SCHOOL_YEAR_ID               1040190 non-null  int64 
 1   STN                          1040190 non-null  object
 2   IDOE_CORPORATION_ID          1040190 non-null  int64 
 3   CORPORATION_NAME             1040190 non-null  object
 4   IDOE_SCHOOL_ID               1040190 non-null  object
 5   SCHOOL_NAME                  1040190 non-null  object
 6   BIRTH_DATE                   1040190 non-null  object
 7   PRIMARY_EXCEPTIONALITY_CODE  1040190 non-null  int64 
 8   PRIMARY_EXCEPTIONALITY       165908 non-null   object
 9   SOCIO_ECON_STATUS            1040190 non-null  object
 10  ETHNICITY                    1040190 non-null  object
dtypes: int64(3), object(8)
memory usage: 87.3+ MB


In [10]:
df1.columns = df1.columns.str.lower()

In [11]:
df1['stn'].duplicated(keep = False).sum()

np.int64(0)

In [12]:
# duplicated_stn = df1.duplicated(keep = False)
# df2 = pd.concat([df1[duplicated_stn], df1[~duplicated_stn]])

### Saving Duplicate stn in excel

In [13]:
# 'stn' is the column containing student identification numbers
#duplicated_stn_rows = df1[df1.duplicated(subset=['stn'], keep=False)]

# Save to Excel
#duplicated_stn_rows.to_excel('duplicated_stn_with_all_fields.xlsx', index=False)

In [14]:
df1.shape

(1040190, 11)

In [15]:
df1.nunique()

,0
school_year_id,1
stn,1040190
idoe_corporation_id,427
corporation_name,427
idoe_school_id,1891
school_name,1808
birth_date,10901
primary_exceptionality_code,17
primary_exceptionality,15
socio_econ_status,3


In [16]:
#df1.describe()

### Calculate the birthdate

In [17]:
from datetime import date
df1['age'] = (date.today().year - pd.to_datetime(df1['birth_date']).dt.year)

### check the age

In [18]:
#df1.age.value_counts()

###Ethnicity algin with fedral requirment.

In [19]:
df1 = df1.replace({'White':'WH7', 'Black':'BL7', 'Hispanic': 'HI7', 'Multiracial': 'MU7', 'Asian' : 'AS7', 'American Indian' : 'AM7', 'Native Hawaiian or Other Pacific Islander' : 'PI7'})

In [20]:
#df1.ethnicity.value_counts()

In [21]:
#Flag SPED eligibility from primary exceptionality column
df1['idea-eligibility'] = np.where(df1['primary_exceptionality'].isnull(), "NonSPED", "SPED")

In [22]:
df1['idea-eligibility'].value_counts()

,count
idea-eligibility,
NonSPED,874282
SPED,165908


In [23]:
df1.shape

(1040190, 13)

In [24]:
print(874282+165908)

1040190


##School Age IDEA enrollment(5 -21)

In [25]:
sch_age = df1[
    (df1['age'] >= 5) &
    (df1['age'] <= 21)
    ]

In [26]:
#sch_age.age.value_counts()

In [27]:
sch_age.dropna(subset = ['ethnicity'], inplace = True)

<ipython-input-27-1e1a79125d14>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sch_age.dropna(subset = ['ethnicity'], inplace = True)


In [28]:
sch_age['ethnicity'] = sch_age['ethnicity'].astype(str)

<ipython-input-28-de393e40504a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sch_age['ethnicity'] = sch_age['ethnicity'].astype(str)


In [29]:
sch_age.columns

Index(['school_year_id', 'stn', 'idoe_corporation_id', 'corporation_name',
       'idoe_school_id', 'school_name', 'birth_date',
       'primary_exceptionality_code', 'primary_exceptionality',
       'socio_econ_status', 'ethnicity', 'age', 'idea-eligibility'],
      dtype='object')

##Total enrollment form PE

In [30]:
pe_total_enrolled = sch_age['idoe_corporation_id'].value_counts().reindex(all_cops, fill_value=0).reset_index()
pe_total_enrolled.columns = ['idoe_corporation_id', 'pe_total_enrolled']
#pe_total_enrolled

Enrollment By Race/Ethnicity from PE

In [31]:
pe_total_enrolled_race = sch_age.groupby('idoe_corporation_id')['ethnicity'].value_counts().unstack(fill_value=0).reindex(all_cops, fill_value=0)
pe_total_enrolled_race = pe_total_enrolled_race.reset_index()
#pe_total_enrolled_race

Merged total enrollment and race enrollment regardles of SPED

In [32]:
# Merge the total enrollment with the race-specific enrollment
pe_merged = pd.merge(pe_total_enrolled, pe_total_enrolled_race, on='idoe_corporation_id', how='left')



In [33]:
pe_merged

,idoe_corporation_id,pe_total_enrolled,AM7,AS7,BL7,HI7,MU7,PI7,WH7
0,-999,0,0,0,0,0,0,0,0
1,15,1356,1,6,7,62,33,0,1247
2,25,1557,5,7,34,177,76,2,1256
3,35,1301,2,12,8,153,25,0,1101
4,125,7453,18,357,625,538,457,4,5454
...,...,...,...,...,...,...,...,...,...
506,9022,193,1,0,78,18,6,0,90
507,9027,113,0,1,53,21,15,0,23
508,9036,54,0,1,13,14,7,0,19
509,9043,286,0,2,43,59,36,0,146


#indicator 9 all other student enrolled which means total enrolled minus target race enrolled from PE file
need to count each race, then substract from total

# AM7 data

In [34]:
am7_enrl = sch_age[sch_age['ethnicity'] == 'AM7']

am7_enrl = am7_enrl.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
am7_enrl.columns = ['idoe_corporation_id', 'pe_am7_enrl']



In [35]:
# Merge the two DataFrames on 'idoe_corporation_id'
merged_enrl_am7 = pd.merge(pe_total_enrolled, am7_enrl, on='idoe_corporation_id', how='left')

# Fill NaN values in 'am7_enrolled' with 0 (for corporations with no AM7 students)
#merged_enrl_am7['pe_am7_enrl'].fillna(0, inplace=True)

# Calculate "All Other" enrollment by subtracting 'am7_enrolled' from 'pe_total_enrolled'
merged_enrl_am7['other_frm_pe_am7'] = merged_enrl_am7['pe_total_enrolled'] - merged_enrl_am7['pe_am7_enrl']

In [36]:
merged_enrl_am7 = merged_enrl_am7.drop(columns = ['pe_am7_enrl', 'pe_total_enrolled'])
merged_enrl_am7['other_frm_pe_am7'] = merged_enrl_am7['other_frm_pe_am7'].astype(int)
#merged_enrl_am7.head()

## merged AM7 data after substracting from total enrollment

In [37]:
pe_merged_am = pd.merge(pe_merged, merged_enrl_am7, on='idoe_corporation_id', how='left')
#pe_merged_am.head()

# AS7 data

In [38]:
as7_enrl = sch_age[sch_age['ethnicity'] == 'AS7']

as7_enrl = as7_enrl.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
as7_enrl.columns = ['idoe_corporation_id', 'pe_as7_enrl']

#merged
merged_as7 = pd.merge(pe_total_enrolled, as7_enrl, on='idoe_corporation_id', how='left')

merged_as7['other_frm_pe_as7'] = merged_as7['pe_total_enrolled'] - merged_as7['pe_as7_enrl']

#drop columns
merged_as7 = merged_as7.drop(columns = ['pe_as7_enrl', 'pe_total_enrolled'])
merged_as7['other_frm_pe_as7'] = merged_as7['other_frm_pe_as7'].astype(int)
pe_merged_as = pd.merge(pe_merged_am, merged_as7, on='idoe_corporation_id', how='left')
#pe_merged_as.head()





#BL7 from other

In [39]:
bl7_enrl = sch_age[sch_age['ethnicity'] == 'BL7']

bl7_enrl = bl7_enrl.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
bl7_enrl.columns = ['idoe_corporation_id', 'pe_bl7_enrl']

#merged
merged_bl7 = pd.merge(pe_total_enrolled, bl7_enrl, on='idoe_corporation_id', how='left')

merged_bl7['other_frm_pe_bl7'] = merged_bl7['pe_total_enrolled'] - merged_bl7['pe_bl7_enrl']

#drop columns
merged_bl7 = merged_bl7.drop(columns = ['pe_bl7_enrl', 'pe_total_enrolled'])
merged_bl7['other_frm_pe_bl7'] = merged_bl7['other_frm_pe_bl7'].astype(int)
pe_merged_bl = pd.merge(pe_merged_as, merged_bl7, on='idoe_corporation_id', how='left')
#pe_merged_bl.head()

#HI7 from others

In [40]:
HI7_enrl = sch_age[sch_age['ethnicity'] == 'HI7']

HI7_enrl = HI7_enrl.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
HI7_enrl.columns = ['idoe_corporation_id', 'pe_HI7_enrl']

#merged
merged_HI7 = pd.merge(pe_total_enrolled, HI7_enrl, on='idoe_corporation_id', how='left')

merged_HI7['other_frm_pe_HI7'] = merged_HI7['pe_total_enrolled'] - merged_HI7['pe_HI7_enrl']

#drop columns
merged_HI7 = merged_HI7.drop(columns = ['pe_HI7_enrl', 'pe_total_enrolled'])
merged_HI7['other_frm_pe_HI7'] = merged_HI7['other_frm_pe_HI7'].astype(int)
pe_merged_HI = pd.merge(pe_merged_bl, merged_HI7, on='idoe_corporation_id', how='left')
pe_merged_HI.head()

,idoe_corporation_id,pe_total_enrolled,AM7,AS7,BL7,HI7,MU7,PI7,WH7,other_frm_pe_am7,other_frm_pe_as7,other_frm_pe_bl7,other_frm_pe_HI7
0,-999,0,0,0,0,0,0,0,0,0,0,0,0
1,15,1356,1,6,7,62,33,0,1247,1355,1350,1349,1294
2,25,1557,5,7,34,177,76,2,1256,1552,1550,1523,1380
3,35,1301,2,12,8,153,25,0,1101,1299,1289,1293,1148
4,125,7453,18,357,625,538,457,4,5454,7435,7096,6828,6915


#other from MU7

In [41]:
MU7_enrl = sch_age[sch_age['ethnicity'] == 'MU7']

MU7_enrl = MU7_enrl.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
MU7_enrl.columns = ['idoe_corporation_id', 'pe_MU7_enrl']

#merged
merged_MU7 = pd.merge(pe_total_enrolled, MU7_enrl, on='idoe_corporation_id', how='left')

merged_MU7['other_frm_pe_MU7'] = merged_MU7['pe_total_enrolled'] - merged_MU7['pe_MU7_enrl']

#drop columns
merged_MU7 = merged_MU7.drop(columns = ['pe_MU7_enrl', 'pe_total_enrolled'])
merged_MU7['other_frm_pe_MU7'] = merged_MU7['other_frm_pe_MU7'].astype(int)
pe_merged_MU = pd.merge(pe_merged_HI, merged_MU7, on='idoe_corporation_id', how='left')
pe_merged_MU.head()

,idoe_corporation_id,pe_total_enrolled,AM7,AS7,BL7,HI7,MU7,PI7,WH7,other_frm_pe_am7,other_frm_pe_as7,other_frm_pe_bl7,other_frm_pe_HI7,other_frm_pe_MU7
0,-999,0,0,0,0,0,0,0,0,0,0,0,0,0
1,15,1356,1,6,7,62,33,0,1247,1355,1350,1349,1294,1323
2,25,1557,5,7,34,177,76,2,1256,1552,1550,1523,1380,1481
3,35,1301,2,12,8,153,25,0,1101,1299,1289,1293,1148,1276
4,125,7453,18,357,625,538,457,4,5454,7435,7096,6828,6915,6996


#Other from PI7

In [42]:
PI7_enrl = sch_age[sch_age['ethnicity'] == 'PI7']

PI7_enrl = PI7_enrl.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
PI7_enrl.columns = ['idoe_corporation_id', 'pe_PI7_enrl']

#merged
merged_PI7 = pd.merge(pe_total_enrolled, PI7_enrl, on='idoe_corporation_id', how='left')

merged_PI7['other_frm_pe_PI7'] = merged_PI7['pe_total_enrolled'] - merged_PI7['pe_PI7_enrl']

#drop columns
merged_PI7 = merged_PI7.drop(columns = ['pe_PI7_enrl', 'pe_total_enrolled'])
merged_PI7['other_frm_pe_PI7'] = merged_PI7['other_frm_pe_PI7'].astype(int)
pe_merged_PI = pd.merge(pe_merged_MU, merged_PI7, on='idoe_corporation_id', how='left')
pe_merged_PI.head()

,idoe_corporation_id,pe_total_enrolled,AM7,AS7,BL7,HI7,MU7,PI7,WH7,other_frm_pe_am7,other_frm_pe_as7,other_frm_pe_bl7,other_frm_pe_HI7,other_frm_pe_MU7,other_frm_pe_PI7
0,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,15,1356,1,6,7,62,33,0,1247,1355,1350,1349,1294,1323,1356
2,25,1557,5,7,34,177,76,2,1256,1552,1550,1523,1380,1481,1555
3,35,1301,2,12,8,153,25,0,1101,1299,1289,1293,1148,1276,1301
4,125,7453,18,357,625,538,457,4,5454,7435,7096,6828,6915,6996,7449


#Other from WH7

In [43]:
WH7_enrl = sch_age[sch_age['ethnicity'] == 'WH7']

WH7_enrl = WH7_enrl.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
WH7_enrl.columns = ['idoe_corporation_id', 'pe_WH7_enrl']

#merged
merged_WH7 = pd.merge(pe_total_enrolled, WH7_enrl, on='idoe_corporation_id', how='left')

merged_WH7['other_frm_pe_WH7'] = merged_WH7['pe_total_enrolled'] - merged_WH7['pe_WH7_enrl']

#drop columns
merged_WH7 = merged_WH7.drop(columns = ['pe_WH7_enrl', 'pe_total_enrolled'])
merged_WH7['other_frm_pe_WH7'] = merged_WH7['other_frm_pe_WH7'].astype(int)
pe_merged_WH = pd.merge(pe_merged_PI, merged_WH7, on='idoe_corporation_id', how='left')

pe_merged_WH.head()

,idoe_corporation_id,pe_total_enrolled,AM7,AS7,BL7,HI7,MU7,PI7,WH7,other_frm_pe_am7,other_frm_pe_as7,other_frm_pe_bl7,other_frm_pe_HI7,other_frm_pe_MU7,other_frm_pe_PI7,other_frm_pe_WH7
0,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,15,1356,1,6,7,62,33,0,1247,1355,1350,1349,1294,1323,1356,109
2,25,1557,5,7,34,177,76,2,1256,1552,1550,1523,1380,1481,1555,301
3,35,1301,2,12,8,153,25,0,1101,1299,1289,1293,1148,1276,1301,200
4,125,7453,18,357,625,538,457,4,5454,7435,7096,6828,6915,6996,7449,1999


#SE file data

In [44]:
se = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/DRF-64 - Brandon Myers Special Education 2025 December Count 01232025.xlsx')

In [45]:
se.shape

(200529, 23)

In [46]:
se.columns = se.columns.str.lower()

In [47]:
duplicated_stn = se.duplicated(keep = False)
se1  = pd.concat([se[duplicated_stn],se[~duplicated_stn]]) # put duplicated on top


In [48]:
#drop duplicate
se1.drop_duplicates(inplace = True)

In [49]:
se1.shape

(200515, 23)

In [50]:
se1.replace('KG - Morning', 'KG', inplace = True)

In [51]:
# columns convert float to integer
cols = ['reporting_corporation', 'legal_corporation', 'primaryexceptionality', 'secondaryexceptionality', 'location', 'adm_type_id']
se1[cols] = se1[cols].astype('Int64')

In [52]:
se1['grade'] = se1['grade'].astype(str)
se1['grade'] = se1['grade'].str.strip()

In [53]:
pd.set_option("display.max_columns", None)

####Excluded from the above list

*   Choice Voucher
*   Identified Unserved NonPublic/Homeschool Only
*   Identified Unserved NonPublic/HomeSchool Only
*   Correctional Facility
*   Parentally placed in Private School

In [54]:
values_to_remove = ['Choice Voucher',
                    'Identified Unserved NonPublic/Homeschool Only',
                    'Identified Unserved NonPublic/HomeSchool Only',
                    'Correctional Facility',
                    'Parentally placed in Private School'
]

se1= se1[~se1['programname'].isin(values_to_remove)]

In [55]:
#se1.programname.value_counts()

In [56]:
se1.replace({'White':'WH7', 'Black':'BL7', 'Hispanic': 'HI7', 'Multiracial': 'MU7', 'Asian' : 'AS7', 'American Indian' : 'AM7', 'Native Hawaiian or Other Pacific Islander' : 'PI7'}, inplace = True)

In [57]:
#se1.ethnicity.value_counts()

####Exclude location: 2, 3, 4, 10

In [58]:
se1 = se1[~se1['location'].isin([2, 3, 4, 10])]

In [59]:
#se1.location.value_counts()

In [60]:
se1.rename(columns={'reporting_corporation' : 'idoe_corporation_id'}, inplace = True)

<ipython-input-60-cc570f1576e9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  se1.rename(columns={'reporting_corporation' : 'idoe_corporation_id'}, inplace = True)


##age 5- 21 or school age, and grade KG - 12

In [61]:
se_sch_age = se1[
    (se1['grade'].isin(['KG', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])) &
    (se1['age_on_01_dec_2024'] >= 5) &
    (se1['age_on_01_dec_2024'] <= 21)
]

In [62]:
#se_sch_age.age_on_01_dec_2024.value_counts()

##SE total count or total Special ed count

In [63]:
se_total_count = se_sch_age['idoe_corporation_id'].value_counts().reindex(all_cops, fill_value=0).reset_index()
se_total_count.columns = ['idoe_corporation_id', 'se_total_enrl']
se_total_count

,idoe_corporation_id,se_total_enrl
0,-999,0
1,15,221
2,25,282
3,35,239
4,125,912
...,...,...
506,9022,54
507,9027,21
508,9036,14
509,9043,49


In [64]:
se_race_count = se_sch_age.groupby('idoe_corporation_id')['ethnicity'].value_counts().unstack(fill_value=0).reindex(all_cops, fill_value=0)
se_race_count = se_race_count.reset_index()





##merged total special ed and count by race/ethnicity of SPED

In [65]:
#se_race_count.head()

In [66]:
merged_se = pd.merge(se_total_count, se_race_count, on='idoe_corporation_id', how='left')
merged_se.head()

,idoe_corporation_id,se_total_enrl,AM7,AS7,BL7,HI7,MU7,PI7,WH7
0,-999,0,0,0,0,0,0,0,0
1,15,221,1,3,2,13,6,0,196
2,25,282,1,0,7,45,14,0,215
3,35,239,1,1,1,21,4,0,211
4,125,912,5,30,121,71,68,0,617


#Count other SPED enrollment (Total count minus target race) -- From SE File

#AM7 data from SE

don't need to write that code each time for 7 races
copy that code and replce it
copy that code paste in the note pad and click on ctrl H replace AM to AS

In [67]:
se_am7 = se_sch_age[se_sch_age['ethnicity'] == 'AM7']
se_am7 = se_am7.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
se_am7.columns = ['idoe_corporation_id', 'se_am7_enrl']

#merged
merged_se_am7 = pd.merge(se_total_count, se_am7, on='idoe_corporation_id', how='left')
merged_se_am7['other_frm_se_am7'] = merged_se_am7['se_total_enrl'] - merged_se_am7['se_am7_enrl']

#drop columns
merged_se_am7 = merged_se_am7.drop(columns = ['se_am7_enrl', 'se_total_enrl'])
merged_se_am7['other_frm_se_am7'] = merged_se_am7['other_frm_se_am7'].astype(int)
se_merged_am = pd.merge(merged_se, merged_se_am7, on='idoe_corporation_id', how='left')
se_merged_am.head()

,idoe_corporation_id,se_total_enrl,AM7,AS7,BL7,HI7,MU7,PI7,WH7,other_frm_se_am7
0,-999,0,0,0,0,0,0,0,0,0
1,15,221,1,3,2,13,6,0,196,220
2,25,282,1,0,7,45,14,0,215,281
3,35,239,1,1,1,21,4,0,211,238
4,125,912,5,30,121,71,68,0,617,907


#AS7 data from SE

In [68]:
se_as7 = se_sch_age[se_sch_age['ethnicity'] == 'AS7']
se_as7 = se_as7.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
se_as7.columns = ['idoe_corporation_id', 'se_as7_enrl']

#merged
merged_se_as7 = pd.merge(se_total_count, se_as7, on='idoe_corporation_id', how='left')
merged_se_as7['other_frm_se_as7'] = merged_se_as7['se_total_enrl'] - merged_se_as7['se_as7_enrl']

#drop columns
merged_se_as7 = merged_se_as7.drop(columns = ['se_as7_enrl', 'se_total_enrl'])
merged_se_as7['other_frm_se_as7'] = merged_se_as7['other_frm_se_as7'].astype(int)
se_merged_as = pd.merge(se_merged_am, merged_se_as7, on='idoe_corporation_id', how='left')
se_merged_as.head()

,idoe_corporation_id,se_total_enrl,AM7,AS7,BL7,HI7,MU7,PI7,WH7,other_frm_se_am7,other_frm_se_as7
0,-999,0,0,0,0,0,0,0,0,0,0
1,15,221,1,3,2,13,6,0,196,220,218
2,25,282,1,0,7,45,14,0,215,281,282
3,35,239,1,1,1,21,4,0,211,238,238
4,125,912,5,30,121,71,68,0,617,907,882


#BL7 data from SE

In [69]:
se_bl7 = se_sch_age[se_sch_age['ethnicity'] == 'BL7']
se_bl7 = se_bl7.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
se_bl7.columns = ['idoe_corporation_id', 'se_bl7_enrl']

#merged
merged_se_bl7 = pd.merge(se_total_count, se_bl7, on='idoe_corporation_id', how='left')
merged_se_bl7['other_frm_se_bl7'] = merged_se_bl7['se_total_enrl'] - merged_se_bl7['se_bl7_enrl']

#drop columns
merged_se_bl7 = merged_se_bl7.drop(columns = ['se_bl7_enrl', 'se_total_enrl'])
merged_se_bl7['other_frm_se_bl7'] = merged_se_bl7['other_frm_se_bl7'].astype(int)
se_merged_bl = pd.merge(se_merged_as, merged_se_bl7, on='idoe_corporation_id', how='left')
se_merged_bl.head()

,idoe_corporation_id,se_total_enrl,AM7,AS7,BL7,HI7,MU7,PI7,WH7,other_frm_se_am7,other_frm_se_as7,other_frm_se_bl7
0,-999,0,0,0,0,0,0,0,0,0,0,0
1,15,221,1,3,2,13,6,0,196,220,218,219
2,25,282,1,0,7,45,14,0,215,281,282,275
3,35,239,1,1,1,21,4,0,211,238,238,238
4,125,912,5,30,121,71,68,0,617,907,882,791


#HI7 from SE

In [70]:
se_hi7 = se_sch_age[se_sch_age['ethnicity'] == 'HI7']
se_hi7 = se_hi7.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
se_hi7.columns = ['idoe_corporation_id', 'se_hi7_enrl']

#merged
merged_se_hi7 = pd.merge(se_total_count, se_hi7, on='idoe_corporation_id', how='left')
merged_se_hi7['other_frm_se_hi7'] = merged_se_hi7['se_total_enrl'] - merged_se_hi7['se_hi7_enrl']

#drop columns
merged_se_hi7 = merged_se_hi7.drop(columns = ['se_hi7_enrl', 'se_total_enrl'])
merged_se_hi7['other_frm_se_hi7'] = merged_se_hi7['other_frm_se_hi7'].astype(int)
se_merged_hi = pd.merge(se_merged_bl, merged_se_hi7, on='idoe_corporation_id', how='left')
se_merged_hi.head()

,idoe_corporation_id,se_total_enrl,AM7,AS7,BL7,HI7,MU7,PI7,WH7,other_frm_se_am7,other_frm_se_as7,other_frm_se_bl7,other_frm_se_hi7
0,-999,0,0,0,0,0,0,0,0,0,0,0,0
1,15,221,1,3,2,13,6,0,196,220,218,219,208
2,25,282,1,0,7,45,14,0,215,281,282,275,237
3,35,239,1,1,1,21,4,0,211,238,238,238,218
4,125,912,5,30,121,71,68,0,617,907,882,791,841


#MU data from SE

In [71]:
se_mu7 = se_sch_age[se_sch_age['ethnicity'] == 'MU7']
se_mu7 = se_mu7.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
se_mu7.columns = ['idoe_corporation_id', 'se_mu7_enrl']

#merged
merged_se_mu7 = pd.merge(se_total_count, se_mu7, on='idoe_corporation_id', how='left')
merged_se_mu7['other_frm_se_mu7'] = merged_se_mu7['se_total_enrl'] - merged_se_mu7['se_mu7_enrl']

#drop columns
merged_se_mu7 = merged_se_mu7.drop(columns = ['se_mu7_enrl', 'se_total_enrl'])
merged_se_mu7['other_frm_se_mu7'] = merged_se_mu7['other_frm_se_mu7'].astype(int)
se_merged_mu = pd.merge(se_merged_hi, merged_se_mu7, on='idoe_corporation_id', how='left')
se_merged_mu.head()


,idoe_corporation_id,se_total_enrl,AM7,AS7,BL7,HI7,MU7,PI7,WH7,other_frm_se_am7,other_frm_se_as7,other_frm_se_bl7,other_frm_se_hi7,other_frm_se_mu7
0,-999,0,0,0,0,0,0,0,0,0,0,0,0,0
1,15,221,1,3,2,13,6,0,196,220,218,219,208,215
2,25,282,1,0,7,45,14,0,215,281,282,275,237,268
3,35,239,1,1,1,21,4,0,211,238,238,238,218,235
4,125,912,5,30,121,71,68,0,617,907,882,791,841,844


#PI data from SE

In [72]:
se_pi7 = se_sch_age[se_sch_age['ethnicity'] == 'PI7']
se_pi7 = se_pi7.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
se_pi7.columns = ['idoe_corporation_id', 'se_pi7_enrl']

#merged
merged_se_pi7 = pd.merge(se_total_count, se_pi7, on='idoe_corporation_id', how='left')
merged_se_pi7['other_frm_se_pi7'] = merged_se_pi7['se_total_enrl'] - merged_se_pi7['se_pi7_enrl']

#drop columns
merged_se_pi7 = merged_se_pi7.drop(columns = ['se_pi7_enrl', 'se_total_enrl'])
merged_se_pi7['other_frm_se_pi7'] = merged_se_pi7['other_frm_se_pi7'].astype(int)
se_merged_pi = pd.merge(se_merged_mu, merged_se_pi7, on='idoe_corporation_id', how='left')
se_merged_pi.head()

,idoe_corporation_id,se_total_enrl,AM7,AS7,BL7,HI7,MU7,PI7,WH7,other_frm_se_am7,other_frm_se_as7,other_frm_se_bl7,other_frm_se_hi7,other_frm_se_mu7,other_frm_se_pi7
0,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,15,221,1,3,2,13,6,0,196,220,218,219,208,215,221
2,25,282,1,0,7,45,14,0,215,281,282,275,237,268,282
3,35,239,1,1,1,21,4,0,211,238,238,238,218,235,239
4,125,912,5,30,121,71,68,0,617,907,882,791,841,844,912


#WH data from SE

In [73]:
se_wh7 = se_sch_age[se_sch_age['ethnicity'] == 'WH7']
se_wh7 = se_wh7.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
se_wh7.columns = ['idoe_corporation_id', 'se_wh7_enrl']

#merged
merged_se_wh7 = pd.merge(se_total_count, se_wh7, on='idoe_corporation_id', how='left')
merged_se_wh7['other_frm_se_wh7'] = merged_se_wh7['se_total_enrl'] - merged_se_wh7['se_wh7_enrl']

#drop columns
merged_se_wh7 = merged_se_wh7.drop(columns = ['se_wh7_enrl', 'se_total_enrl'])
merged_se_wh7['other_frm_se_wh7'] = merged_se_wh7['other_frm_se_wh7'].astype(int)
se_merged_wh = pd.merge(se_merged_pi, merged_se_wh7, on='idoe_corporation_id', how='left')
se_merged_wh.head()



,idoe_corporation_id,se_total_enrl,AM7,AS7,BL7,HI7,MU7,PI7,WH7,other_frm_se_am7,other_frm_se_as7,other_frm_se_bl7,other_frm_se_hi7,other_frm_se_mu7,other_frm_se_pi7,other_frm_se_wh7
0,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,15,221,1,3,2,13,6,0,196,220,218,219,208,215,221,25
2,25,282,1,0,7,45,14,0,215,281,282,275,237,268,282,67
3,35,239,1,1,1,21,4,0,211,238,238,238,218,235,239,28
4,125,912,5,30,121,71,68,0,617,907,882,791,841,844,912,295


#merged PE file to SE file for indicator 9 caculation

In [74]:
se_merged_wh.head()

,idoe_corporation_id,se_total_enrl,AM7,AS7,BL7,HI7,MU7,PI7,WH7,other_frm_se_am7,other_frm_se_as7,other_frm_se_bl7,other_frm_se_hi7,other_frm_se_mu7,other_frm_se_pi7,other_frm_se_wh7
0,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,15,221,1,3,2,13,6,0,196,220,218,219,208,215,221,25
2,25,282,1,0,7,45,14,0,215,281,282,275,237,268,282,67
3,35,239,1,1,1,21,4,0,211,238,238,238,218,235,239,28
4,125,912,5,30,121,71,68,0,617,907,882,791,841,844,912,295


##Rename race/ethnicity columns before merging to PE file

In [75]:
se_merged = se_merged_wh.rename(columns = {'AM7': 'SE_AM7', 'AS7': 'SE_AS7', 'BL7': 'SE_BL7', 'HI7': 'SE_HI7', 'MU7': 'SE_MU7', 'PI7': 'SE_PI7', 'WH7': 'SE_WH7'})

In [76]:
#se_merged_wh.head()

In [77]:
all_merged = pd.merge(pe_merged_WH, se_merged, on='idoe_corporation_id', how='left')

In [78]:
all_merged.head()

,idoe_corporation_id,pe_total_enrolled,AM7,AS7,BL7,HI7,MU7,PI7,WH7,other_frm_pe_am7,other_frm_pe_as7,other_frm_pe_bl7,other_frm_pe_HI7,other_frm_pe_MU7,other_frm_pe_PI7,other_frm_pe_WH7,se_total_enrl,SE_AM7,SE_AS7,SE_BL7,SE_HI7,SE_MU7,SE_PI7,SE_WH7,other_frm_se_am7,other_frm_se_as7,other_frm_se_bl7,other_frm_se_hi7,other_frm_se_mu7,other_frm_se_pi7,other_frm_se_wh7
0,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,15,1356,1,6,7,62,33,0,1247,1355,1350,1349,1294,1323,1356,109,221,1,3,2,13,6,0,196,220,218,219,208,215,221,25
2,25,1557,5,7,34,177,76,2,1256,1552,1550,1523,1380,1481,1555,301,282,1,0,7,45,14,0,215,281,282,275,237,268,282,67
3,35,1301,2,12,8,153,25,0,1101,1299,1289,1293,1148,1276,1301,200,239,1,1,1,21,4,0,211,238,238,238,218,235,239,28
4,125,7453,18,357,625,538,457,4,5454,7435,7096,6828,6915,6996,7449,1999,912,5,30,121,71,68,0,617,907,882,791,841,844,912,295


#Indicator 9 calculation

Target Risk : Black student with disability / Black student enrolled

Other Risk: All other Students with disability  divided by All other Students enrolled.

Risk ratio =  Target Risk divided by Other Risk

Black student with disability – N Size =  15

Black student enrolled Cell Size = 15

All other Students with disability  N Size = 15

All other Students enrolled. cell Size = 15




In [79]:
all_merged.shape

(511, 31)

##Risk ratio American Indian (AM7)

In [80]:
all_merged['risk_ratio_am7'] = (all_merged['SE_AM7'] * all_merged['other_frm_pe_am7']) / (all_merged['AM7'] * all_merged['other_frm_se_am7'])
all_merged['risk_ratio_am7'] = all_merged['risk_ratio_am7'].replace([np.inf, -np.inf], np.nan)

##Risk ratio Asian (AS7)

In [81]:
all_merged['risk_ratio_as7'] = (all_merged['SE_AS7'] * all_merged['other_frm_pe_as7']) / (all_merged['AS7'] * all_merged['other_frm_se_as7'])
all_merged['risk_ratio_as7'] = all_merged['risk_ratio_as7'].replace([np.inf, -np.inf], np.nan)

##Risk ratio Black(BL7)

In [82]:
all_merged['risk_ratio_bl7'] = (all_merged['SE_BL7'] * all_merged['other_frm_pe_bl7']) / (all_merged['BL7'] * all_merged['other_frm_se_bl7'])
all_merged['risk_ratio_bl7'] = all_merged['risk_ratio_bl7'].replace([np.inf, -np.inf], np.nan)

##Risk ratio Hispanic(HI7)

In [83]:
all_merged['risk_ratio_hi7'] = (all_merged['SE_HI7'] * all_merged['other_frm_pe_HI7']) / (all_merged['HI7'] * all_merged['other_frm_se_hi7'])
all_merged['risk_ratio_hi7'] = all_merged['risk_ratio_hi7'].replace([np.inf, -np.inf], np.nan)

##Risk ratio Multiracial (Mu7)

In [84]:
all_merged['risk_ratio_mu7'] = (all_merged['SE_MU7'] * all_merged['other_frm_pe_MU7']) / (all_merged['MU7'] * all_merged['other_frm_se_mu7'])
all_merged['risk_ratio_mu7'] = all_merged['risk_ratio_mu7'].replace([np.inf, -np.inf], np.nan)

##Risk ratio Native Hawaiian or Other Pacific Islander(PI7)

In [85]:
all_merged['risk_ratio_pi7'] = (all_merged['SE_PI7'] * all_merged['other_frm_pe_PI7']) / (all_merged['PI7'] * all_merged['other_frm_se_pi7'])
all_merged['risk_ratio_pi7'] = all_merged['risk_ratio_pi7'].replace([np.inf, -np.inf], np.nan)

##Risk ratio White(WH)

In [86]:
all_merged['risk_ratio_wh7'] = (all_merged['SE_WH7'] * all_merged['other_frm_pe_WH7']) / (all_merged['WH7'] * all_merged['other_frm_se_wh7'])
all_merged['risk_ratio_wh7'] = all_merged['risk_ratio_wh7'].replace([np.inf, -np.inf], np.nan)

In [87]:
all_merged.head()

,idoe_corporation_id,pe_total_enrolled,AM7,AS7,BL7,HI7,MU7,PI7,WH7,other_frm_pe_am7,other_frm_pe_as7,other_frm_pe_bl7,other_frm_pe_HI7,other_frm_pe_MU7,other_frm_pe_PI7,other_frm_pe_WH7,se_total_enrl,SE_AM7,SE_AS7,SE_BL7,SE_HI7,SE_MU7,SE_PI7,SE_WH7,other_frm_se_am7,other_frm_se_as7,other_frm_se_bl7,other_frm_se_hi7,other_frm_se_mu7,other_frm_se_pi7,other_frm_se_wh7,risk_ratio_am7,risk_ratio_as7,risk_ratio_bl7,risk_ratio_hi7,risk_ratio_mu7,risk_ratio_pi7,risk_ratio_wh7
0,-999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15,1356,1,6,7,62,33,0,1247,1355,1350,1349,1294,1323,1356,109,221,1,3,2,13,6,0,196,220,218,219,208,215,221,25,6.159091,3.096330,1.759948,1.304435,1.118816,NaN,0.685293
2,25,1557,5,7,34,177,76,2,1256,1552,1550,1523,1380,1481,1555,301,282,1,0,7,45,14,0,215,281,282,275,237,268,282,67,1.104626,0.000000,1.140214,1.480369,1.017969,0.0,0.769025
3,35,1301,2,12,8,153,25,0,1101,1299,1289,1293,1148,1276,1301,200,239,1,1,1,21,4,0,211,238,238,238,218,235,239,28,2.728992,0.451331,0.679097,0.722792,0.868766,NaN,1.368885
4,125,7453,18,357,625,538,457,4,5454,7435,7096,6828,6915,6996,7449,1999,912,5,30,121,71,68,0,617,907,882,791,841,844,912,295,2.277043,0.676080,1.671177,1.085106,1.233389,0.0,0.766586


In [88]:
#all_merged.to_excel('all_merged_indicator_9_calc.xlsx', index = False)

#Data extraction for indicator 10 calculation

Target risk : Black student with ED divided by black student enrolled

Other risk : All other identified as ED divided by All other student enrolled

Target risk : Black student with ED divided by black student enrolled

Other risk : All other identified as ED divided by All other student enrolled

Six specific disabilities:
1.       Autism
2.       Emotional Disturbance(ED)
3.       Intellectual disability
4.       Other health Impairments
5.       Specific Learning Disability and
6.     Speech or language impairment

need to extract each specific disability with each target race. that data need to mege with pe_merged_WH


Run the disability code file, merge with se file to get disability code

In [89]:
disability = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Disability_codes.xlsx')

In [90]:
#disability.head(20)

Converted full time and part time disability to Emotional disability
mild, moderate, and severe intellectual disability to intellectual disability


In [91]:
disability.replace('Emotional Disability (Full-Time)', 'Emotional Disturbance', inplace = True)
disability.replace('Emotional Disability (Part-Time)', 'Emotional Disturbance', inplace = True)

In [92]:
disability.replace('Mild Intellectual Disability', 'Intellectual Disability', inplace = True)
disability.replace('Moderate Intellectual Disability', 'Intellectual Disability', inplace = True)
disability.replace('Severe Intellectual Disability', 'Intellectual Disability', inplace = True)

In [93]:
#disability.head(20)

In [94]:
se_dis_sch_age = se_sch_age.merge(disability, right_on = 'Code', left_on = 'primaryexceptionality', how = 'left')

In [95]:
se_dis_sch_age.columns

Index(['schoolyear', 'submissionwindowid', 'idoe_corporation_id',
       'reporting_corporation_name', 'legal_corporation',
       'legal_corporation_name', 'idoe_school_id', 'school_name', 'stn',
       'gender', 'ethnicity', 'birth_date', 'age_on_01_dec_2024', 'grade',
       'programname', 'primaryexceptionality', 'secondaryexceptionality',
       'primaryfundinglevel', 'secondaryfundinglevel', 'ideaeligibility',
       'socio_econ_status_category', 'location', 'adm_type_id', 'Code',
       'Disability'],
      dtype='object')

In [97]:
se_dis_sch_age.Disability.value_counts()

,count
Disability,
Specific Learning Disability,51734
Other Health Impairment,31247
Speech/Language Impairment,30726
Autism Spectrum Disorder,19928
Developmental Delay,11909
Intellectual Disability,11885
Emotional Disturbance,10776
Deaf or Hard of Hearing,2135
Multiple Disabilities,1592


In [98]:
from itertools import product
ETHNICITY_CODES = {
    'AM7': 'American Indian',
    'HI7': 'Hispanic',
    'AS7': 'Asian',
    'MU7': 'Multiracial',
    'BL7': 'Black',
    'WH7': 'White',
    'PI7': 'Other Pacific Islander'
}

DISABILITY_TYPES = [
    'Autism Spectrum Disorder',
    'Emotional Disturbance',
    'Intellectual Disability',
    'Other Health Impairment',
    'Specific Learning Disability',
    'Speech/Language Impairment'
]



# 3. Get unique corporation IDs
corporation_ids = se_dis_sch_age['idoe_corporation_id'].unique()

# 4. risk ratio function for corporation-level calculation
def calculate_corporation_rr(corp_id, target_ethnicity, target_disability):
    """Calculate risk ratio for a specific corporation"""
    # Filter data for this corporation
    corp_sch_age = sch_age[sch_age['idoe_corporation_id'] == corp_id]
    corp_disabilities = se_dis_sch_age[se_dis_sch_age['idoe_corporation_id'] == corp_id]

    # Get sch_age counts
    B = corp_sch_age[corp_sch_age['ethnicity'] == target_ethnicity].shape[0]
    D = corp_sch_age[corp_sch_age['ethnicity'] != target_ethnicity].shape[0]

    # Get disability counts
    A = corp_disabilities[(corp_disabilities['ethnicity'] == target_ethnicity) &
                         (corp_disabilities['Disability'] == target_disability)].shape[0]

    C = corp_disabilities[(corp_disabilities['ethnicity'] != target_ethnicity) &
                         (corp_disabilities['Disability'] == target_disability)].shape[0]

    # Calculate risk ratio
    try:
        return (A * D) / (B * C) if (B * C) != 0 else float('nan')
    except:
        return float('nan')

# 5. Calculate all combinations for all corporations
results = []
for corp_id in corporation_ids:
    for eth_code, disability in product(ETHNICITY_CODES.keys(), DISABILITY_TYPES):
        rr = calculate_corporation_rr(corp_id, eth_code, disability)
        results.append({
            'Corporation_ID': corp_id,
            'Ethnicity_Code': eth_code,
            'Ethnicity': ETHNICITY_CODES[eth_code],
            'Disability': disability,
            'Risk_Ratio': rr
        })

# 6. Create results DataFrame
results_df = pd.DataFrame(results)

# 7. Save comprehensive results
results_df.to_csv('indicator_10_risk_ratios.csv', index=False)

# 8. Create a summary pivot table
pivot_summary = results_df.pivot_table(
    index=['Corporation_ID', 'Ethnicity'],
    columns='Disability',
    values='Risk_Ratio'
)

# 9. Save and display summary
pivot_summary.to_csv('indicator_10_risk_ratios_summary.csv')
print("Top 10 Corporation Risk Ratios:")
display(pivot_summary.head())


Top 10 Corporation Risk Ratios:


Disability                      Autism Spectrum Disorder  \
Corporation_ID Ethnicity                                   
15             American Indian                  0.000000   
               Asian                            0.000000   
               Black                            0.000000   
               Hispanic                         2.318996   
               Multiracial                      0.000000   

Disability                      Emotional Disturbance  \
Corporation_ID Ethnicity                                
15             American Indian                    0.0   
               Asian                              0.0   
               Black                              0.0   
               Hispanic                           0.0   
               Multiracial                        0.0   

Disability                      Intellectual Disability  \
Corporation_ID Ethnicity                                  
15             American Indian                 0.000000   
               Asian                          10.714286   
               Black                           0.000000   
               Hispanic                        0.993856   
               Multiracial                     1.909091   

Disability                      Other Health Impairment  \
Corporation_ID Ethnicity                                  
15             American Indian                      0.0   
               Asian                                0.0   
               Black                                0.0   
               Hispanic                             0.0   
               Multiracial                          0.0   

Disability                      Specific Learning Disability  \
Corporation_ID Ethnicity                                       
15             American Indian                     31.511628   
               Asian                                0.000000   
               Black                                0.000000   
               Hispanic                             1.527144   
               Multiracial                          1.909091   

Disability                      Speech/Language Impairment  
Corporation_ID Ethnicity                                    
15             American Indian                    0.000000  
               Asian                              0.000000  
               Black                              2.350174  
               Hispanic                           1.922326  
               Multiracial                        0.488914

In [99]:
#se_am_autism = se_dis_sch_age[se_dis_sch_age['ethnicity'] == 'AM7' and se_dis_sch_age['Disability'] == 'Autism Spectrum Disorder']
# se_am7 = se_am7.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
# se_am7.columns = ['idoe_corporation_id', 'se_am7_enrl']

# #merged
# merged_se_am7 = pd.merge(se_total_count, se_am7, on='idoe_corporation_id', how='left')
# merged_se_am7['other_frm_se_am7'] = merged_se_am7['se_total_enrl'] - merged_se_am7['se_am7_enrl']

# #drop columns
# merged_se_am7 = merged_se_am7.drop(columns = ['se_am7_enrl', 'se_total_enrl'])
# merged_se_am7['other_frm_se_am7'] = merged_se_am7['other_frm_se_am7'].astype(int)
# se_merged_am = pd.merge(merged_se, merged_se_am7, on='idoe_corporation_id', how='left')
# se_merged_am.head()

In [100]:
se_sch_age.head()

,schoolyear,submissionwindowid,idoe_corporation_id,reporting_corporation_name,legal_corporation,legal_corporation_name,idoe_school_id,school_name,stn,gender,ethnicity,birth_date,age_on_01_dec_2024,grade,programname,primaryexceptionality,secondaryexceptionality,primaryfundinglevel,secondaryfundinglevel,ideaeligibility,socio_econ_status_category,location,adm_type_id
14465,2025,31,370,Flat Rock-Hawcreek School Corp,370,Flat Rock-Hawcreek School Corp,0409,Hope Elementary School,T12427090,Male,WH7,2018-06-30,6,KG,Regular Classroom (80% or more),15,9,1,3.0,1,Free/Reduced price meals,<NA>,1
24022,2025,31,1010,Greater Clark County Schools,1010,Greater Clark County Schools,0883,River Valley Middle School,080117099,Male,HI7,2011-09-08,13,07,Separate Class (less than 40%),17,<NA>,2,NaN,1,Free/Reduced price meals,<NA>,1
31937,2025,31,1875,Delaware Community School Corp,1885,Wes-Del Community Schools,1367,Delta Middle School,863313586,Male,WH7,2010-06-26,14,07,Regular Classroom (80% or more),7,<NA>,2,NaN,1,Paid meals,<NA>,1
32227,2025,31,1885,Wes-Del Community Schools,1885,Wes-Del Community Schools,1372,Wes-Del Middle/Senior High School,863313586,Male,WH7,2010-06-26,14,07,Regular Classroom (80% or more),7,<NA>,2,NaN,1,Paid meals,<NA>,1
45081,2025,31,2455,Southeast Fountain School Corp,2455,Southeast Fountain School Corp,2021,Southeast Fountain Elementary,872919029,Female,WH7,2013-11-12,11,05,Resource Room (40% - 79%),7,17,2,NaN,1,Free/Reduced price meals,<NA>,1
